# Test evaluations

In [1]:
from os.path import realpath
from pathlib import Path
import pandas as pd
import itertools
import json
import shutil

def load_json(path):
    with open(path) as f:
        cfg = json.load(f)
    return cfg

RUNS_DIR = Path(realpath('.')) / 'runs'
if not 'domain-adaptation/runs' in str(RUNS_DIR):
    RUNS_DIR = Path(realpath('.')).parent / 'runs'
assert('domain-adaptation/runs' in str(RUNS_DIR))

In [119]:
def load_cls_rep_paths_new(
    method,
    experiment_id,
    prefix=None,
    config_key=None,
    config_value=None,
):
    d = RUNS_DIR / method / experiment_id
    paths = []
    for item in d.glob('*'):
        if (item.is_dir()):
            if config_key:
                if not (config_value == load_json(item / 'config.json')[config_key]):
                    continue
            if prefix:
                if not item.name.startswith(prefix):
                    continue
            paths.append( item / 'report.json' ) 
    return sorted(paths)

def get_score_new(
    report_paths,
    metric:str='precision',
    avg_types=['macro avg', 'weighted avg'],
    map_col_name= lambda n: n
):
    reports = [ pd.read_json(p) for p in report_paths ]
    score = pd.DataFrame(
        [[r[avgt][metric] for avgt in avg_types]
         for r in reports
        ],
        columns = [ map_col_name(avgt) for avgt in avg_types]
    ) 
    return score

def get_score_combinations_new(
    method,
    experiment_id,
    config_key=None,
    config_value=None,
    domains=['A','W','D'], 
    metric:str='recall',
    avg_types=['weighted avg'],
):
    combos = [c for c in itertools.product(domains, repeat=2) if c[0] != c[1]]
    scores = [
        get_score_new(
            report_paths=load_cls_rep_paths_new(method, experiment_id, '{}{}'.format(c[0],c[1]), config_key, config_value),
            metric=metric,
            avg_types=avg_types,
            map_col_name=lambda n: '{}->{}'.format(c[0],c[1]),
        )
        for c in combos
    ]
    return pd.concat(scores, sort=False)


def load_cls_rep_paths(
    suffix:str, 
    runs_dir:Path=RUNS_DIR, 
    from_date:str='19700101000000', 
    to_date:str='30001010000000'
):
    assert(len(from_date)==14 and len(to_date)==14)
    return sorted([
        item / 'report.json' 
        for item in runs_dir.glob('*') 
        if item.is_dir() 
        and item.name.endswith(suffix)
        and int(item.name.split('_')[0]) >= int(from_date)
        and int(item.name.split('_')[0]) <= int(to_date)
    ])

def get_score(
    suffix:str, 
    runs_dir:Path=RUNS_DIR, 
    metric:str='precision',
    avg_types=['macro avg', 'weighted avg'],
    from_date:str='19700101000000', 
    to_date:str='30001010000000',
    map_col_name= lambda n: n
):
    report_paths = load_cls_rep_paths(suffix, RUNS_DIR, from_date, to_date)
    reports = [ pd.read_json(p) for p in report_paths ]
    score = pd.DataFrame(
        [[r[avgt][metric] for avgt in avg_types]
         for r in reports
        ],
        columns = [ map_col_name(avgt) for avgt in avg_types]
    ) 
    return score


def get_score_combinations(
    suffix:str, 
    domains=['A','W','D'], 
    runs_dir:Path=RUNS_DIR, 
    metric:str='accuracy',
    avg_types=['macro avg', 'weighted avg'],
    from_date:str='19700101000000', 
    to_date:str='30001010000000',
):
    combos = [c for c in itertools.product(domains, repeat=2) if c[0] != c[1]]
    scores = [
        get_score(
            suffix='{}_{}_{}'.format(c[0],c[1],suffix),
            runs_dir=runs_dir,
            metric=metric,
            avg_types=avg_types,
            from_date=from_date,
            to_date=to_date,
            map_col_name=lambda n: '{}->{}'.format(c[0],c[1]),
        )
        for c in combos
    ]
    return pd.concat(scores, sort=False)
    

## Tune source only
In this experiment, we tune a VGG16-network pretrained on ImageNet with all available source data.
The target data is used for validation (during training) and test.

In [120]:
# without augmentation
# tune_source_no_aug_scores = get_score_combinations(
#     suffix='tune_source', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191014123846',
#     to_date='20191014162536'
# )*100
# tune_source_no_aug_scores.describe().T

tune_source_no_aug_scores = get_score_combinations_new(
    method='tune_source',
    experiment_id='tune_source_no_aug',
)*100
tune_source_no_aug_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,52.656250,1.250000,51.093750,51.562500,53.125000,53.750000,53.750000
A->D,5.0,56.964286,3.209598,53.571429,53.571429,58.035714,59.226190,60.416667
W->A,5.0,42.341867,1.474058,40.549699,41.114458,42.846386,43.072289,44.126506
W->D,5.0,98.571429,0.572482,97.916667,98.214286,98.511905,98.809524,99.404762
D->A,5.0,36.844880,2.337159,33.471386,35.617470,37.688253,37.951807,39.495482
D->W,5.0,91.031250,2.005851,87.968750,90.468750,91.093750,92.656250,92.968750


In [121]:
# with augmentation
# tune_source_scores = get_score_combinations(
#     suffix='tune_source', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191022103424',
#     to_date='20191022142437'
# )*100
# tune_source_scores.describe().T

tune_source_scores = get_score_combinations_new(
    method='tune_source',
    experiment_id='tune_source_with_aug',
)*100
tune_source_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,55.343750,1.856681,52.812500,53.906250,56.562500,56.562500,56.875000
A->D,5.0,59.107143,1.691461,57.738095,58.035714,58.035714,60.119048,61.607143
W->A,5.0,40.911145,2.005155,38.441265,39.721386,40.662651,42.206325,43.524096
W->D,5.0,98.750000,0.678676,97.916667,98.511905,98.511905,99.107143,99.702381
D->A,5.0,40.820783,1.713495,38.177711,40.173193,41.340361,41.867470,42.545181
D->W,5.0,93.625000,1.202212,91.718750,93.281250,93.906250,94.531250,94.687500


## Tune source and target

In [122]:
# tune_both_scores = get_score_combinations(
#     suffix='tune_target', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191023072318',
#     to_date='20191023123426'
# )*100
# tune_both_scores.describe().T

tune_both_scores = get_score_combinations_new(
    method='tune_target',
    experiment_id='tune_target_with_aug',
)*100
tune_both_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,60.062500,3.517534,55.156250,57.656250,61.406250,62.968750,63.125000
A->D,5.0,65.535714,3.102955,61.011905,64.285714,65.773810,67.559524,69.047619
W->A,5.0,51.438253,1.755988,50.000000,50.150602,51.280120,51.393072,54.367470
W->D,5.0,99.226190,0.580166,98.511905,98.809524,99.404762,99.404762,100.000000
D->A,5.0,52.771084,2.706081,49.284639,51.844880,51.920181,54.480422,56.325301
D->W,5.0,94.562500,2.707231,89.843750,95.156250,95.156250,96.093750,96.562500


## Multitask

In [123]:
multitask_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_with_batchnorn_alpha_0',
)*100
multitask_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,81.687500,1.937122,78.750000,81.250000,81.562500,83.281250,83.593750
A->D,5.0,80.233236,3.690092,75.510204,78.425656,80.174927,81.632653,85.422741
W->A,5.0,62.907588,1.666200,60.668670,62.509391,62.697220,63.373403,65.289256
W->D,5.0,96.326531,1.907338,94.169096,95.043732,95.918367,97.667638,98.833819
D->A,5.0,62.148760,1.908953,58.903080,62.021037,62.960180,63.223140,63.636364
D->W,5.0,93.375000,1.063189,92.031250,92.500000,93.750000,94.062500,94.531250


In [124]:
multitask_from_feat_scores = get_score_combinations_new(
    method='multitask',
    experiment_id='multitask_from_feat',
)*100
multitask_from_feat_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.343750,1.608692,75.312500,75.937500,78.125000,78.437500,78.906250
A->D,5.0,77.201166,2.065655,74.635569,75.510204,77.842566,78.425656,79.591837
W->A,5.0,61.600301,1.392823,59.504132,61.495116,61.570248,62.058603,63.373403
W->D,5.0,93.002915,2.507967,90.670554,91.253644,93.002915,93.002915,97.084548
D->A,5.0,62.231405,0.682726,61.570248,61.758077,62.021037,62.546957,63.260706
D->W,5.0,91.343750,0.831215,90.468750,90.781250,91.093750,91.875000,92.500000


## CCSA

__Run 1__: The were some severe stability issues when introducting the CSA loss (alpha 0.25 as done by original author).
Employed parameters:
- alpha=0.01 
- freeze_base=true

In [125]:
# tune_ccsa_scores = get_score_combinations(
#     suffix='ccsa', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191025141713',
#     to_date='20191025170306'
# )*100
# tune_ccsa_scores.describe().T

tune_ccsa_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_without_batchnorn',
)*100
tune_ccsa_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,75.375000,3.392101,73.125000,73.125000,74.218750,75.156250,81.250000
A->D,5.0,77.259475,3.873274,73.760933,74.927114,75.218659,79.300292,83.090379
W->A,5.0,58.790383,1.411842,57.287754,57.325319,59.316304,59.616829,60.405710
W->D,5.0,93.002915,2.803992,89.795918,91.836735,92.419825,93.586006,97.376093
D->A,5.0,58.827949,1.546368,57.550714,57.625845,58.414726,59.241172,61.307288
D->W,5.0,92.312500,1.955935,89.687500,91.406250,92.656250,92.812500,95.000000


We observe a large performance increment for the domain adaptations where domains are far apart. However, for similar domains (W,D)

__Run 3__ multi-task learning (ccsa code with alpha=0)

In [127]:
# multitask_scores = get_score_combinations(
#     suffix='ccsa', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191101173414',
#     to_date='20191101191220'
# )*100
# multitask_scores.describe().T

multitask_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_with_batchnorn_alpha_0',
)*100
multitask_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,81.687500,1.937122,78.750000,81.250000,81.562500,83.281250,83.593750
A->D,5.0,80.233236,3.690092,75.510204,78.425656,80.174927,81.632653,85.422741
W->A,5.0,62.907588,1.666200,60.668670,62.509391,62.697220,63.373403,65.289256
W->D,5.0,96.326531,1.907338,94.169096,95.043732,95.918367,97.667638,98.833819
D->A,5.0,62.148760,1.908953,58.903080,62.021037,62.960180,63.223140,63.636364
D->W,5.0,93.375000,1.063189,92.031250,92.500000,93.750000,94.062500,94.531250


__Run 4__ ccsa with alpha = 0.25

In [128]:
# ccsa_scores = get_score_combinations(
#     suffix='ccsa', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191101150707',
#     to_date='20191101171905'
# )*100
# ccsa_scores.describe().T

ccsa_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_with_batchnorn_alpha_0.25',
)*100
ccsa_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,82.500000,2.715335,78.750000,80.625000,83.750000,84.062500,85.312500
A->D,5.0,82.274052,3.614455,77.259475,80.174927,83.381924,83.965015,86.588921
W->A,5.0,59.939895,2.546253,56.235913,59.429001,60.142750,60.555973,63.335838
W->D,5.0,93.236152,2.519801,89.212828,92.419825,94.169096,95.043732,95.335277
D->A,5.0,60.112697,1.915190,58.302029,58.377160,59.879790,61.232156,62.772352
D->W,5.0,92.437500,2.334879,88.437500,92.656250,92.968750,93.750000,94.375000


In [129]:
# ccsa_uneven_scores = get_score_combinations(
#     suffix='ccsa', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191104152050',
#     to_date='20191104185818'
# )*100
# ccsa_uneven_scores.describe().T

ccsa_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_uneven',
)*100
ccsa_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,83.468750,1.463254,81.875000,82.656250,83.437500,83.593750,85.781250
A->D,5.0,82.915452,2.245087,79.591837,82.215743,82.798834,84.839650,85.131195
W->A,5.0,60.924117,3.093009,57.362885,57.963937,62.396694,62.471826,64.425244
W->D,5.0,93.935860,1.654374,92.128280,93.002915,93.294461,95.043732,96.209913
D->A,5.0,60.826446,2.820359,58.978212,59.128475,60.030053,60.217881,65.777611
D->W,5.0,92.531250,2.615496,89.062500,91.406250,92.187500,94.062500,95.937500


In [130]:
# ccsa_resnet_uneven_scores = get_score_combinations(
#     suffix='ccsa', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191106083058',
#     to_date='20191106144631'
# )*100
# ccsa_resnet_uneven_scores.describe().T

ccsa_resnet_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_resnet_uneven',
)*100
ccsa_resnet_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,86.687500,1.843485,84.375000,85.312500,86.875000,88.125000,88.750000
A->D,6.0,84.985423,1.697484,83.090379,83.746356,84.693878,86.078717,87.463557
W->A,5.0,66.468820,2.010838,64.312547,64.800902,66.378663,67.693464,69.158527
W->D,5.0,93.352770,2.055342,89.795918,93.586006,93.877551,94.752187,94.752187
D->A,6.0,63.479840,4.039071,56.386176,62.190083,64.406461,66.115702,67.543201
D->W,5.0,92.187500,1.184824,90.937500,91.406250,91.875000,92.812500,93.906250


### CCSA from features
Testing if batch size has an impact on CCSA (like seems to have on DAGE)

In [132]:
ccsa_from_feat_batch_size_16_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    config_key='batch_size',
    config_value=16,
)*100
ccsa_from_feat_batch_size_16_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.250000,1.810074,76.875000,77.968750,79.531250,80.781250,81.093750
A->D,5.0,78.483965,1.641480,76.093294,77.842566,78.717201,79.300292,80.466472
W->A,5.0,59.556724,0.794051,58.602554,58.940646,59.654395,60.030053,60.555973
W->D,5.0,90.029155,3.433556,86.880466,87.172012,89.212828,91.836735,95.043732
D->A,5.0,58.767844,2.074398,56.574005,57.175056,58.189331,60.931630,60.969196
D->W,5.0,90.531250,0.921743,89.687500,89.687500,90.468750,90.937500,91.875000


In [133]:
ccsa_from_feat_batch_size_32_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    config_key='batch_size',
    config_value=32,
)*100
ccsa_from_feat_batch_size_32_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.312500,1.588072,77.343750,78.437500,78.906250,80.937500,80.937500
A->D,5.0,78.075802,2.494373,75.801749,76.967930,76.967930,78.425656,82.215743
W->A,5.0,59.474080,1.356378,58.377160,58.452292,59.090909,59.767092,61.682945
W->D,5.0,90.145773,2.156252,88.338192,89.212828,89.504373,89.795918,93.877551
D->A,5.0,59.120962,1.775632,56.048084,59.203606,59.804658,60.142750,60.405710
D->W,5.0,90.625000,1.076879,89.218750,90.312500,90.468750,90.937500,92.187500


In [16]:
ccsa_from_feat_batch_size_64_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    avg_types=['macro avg'],
    config_key='batch_size',
    config_value=64,
)*100
ccsa_from_feat_batch_size_64_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.708751,2.307348,76.920129,78.636711,79.136540,80.872306,82.978071
A->D,5.0,79.141243,3.449395,74.629593,76.648884,80.174678,81.093436,83.159627
W->A,5.0,61.356650,1.105851,59.478968,61.317171,61.692201,62.124731,62.170181
W->D,5.0,88.773578,2.190124,85.960699,88.369675,88.541375,88.899718,92.096425
D->A,5.0,62.502549,1.694606,60.939045,61.115881,61.921053,63.746415,64.790349
D->W,5.0,90.691934,2.119183,88.559343,89.346390,89.796408,92.091719,93.665811


In [134]:
ccsa_from_feat_batch_size_128_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    config_key='batch_size',
    config_value=128,
)*100
ccsa_from_feat_batch_size_128_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,78.000000,1.606414,75.625000,77.500000,78.281250,78.593750,80.000000
A->D,5.0,76.151603,3.364790,71.720117,75.218659,75.801749,76.967930,81.049563
W->A,5.0,57.978963,2.099753,55.747558,55.935387,58.264463,59.504132,60.443276
W->D,5.0,87.696793,2.214592,85.714286,86.297376,86.880466,88.338192,91.253644
D->A,5.0,57.926371,1.209710,56.085650,57.325319,58.564989,58.640120,59.015778
D->W,5.0,88.156250,2.789402,86.093750,86.718750,86.875000,88.125000,92.968750


In [135]:
ccsa_from_feat_batch_size_256_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    config_key='batch_size',
    config_value=256,
)*100
ccsa_from_feat_batch_size_256_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,10.0,77.000000,2.086976,73.593750,75.742188,77.421875,78.671875,79.375000
A->D,10.0,76.384840,1.677617,72.886297,75.801749,76.239067,77.623907,78.717201
W->A,10.0,53.621337,3.489199,47.107438,52.037941,53.888054,56.414350,58.226897
W->D,10.0,86.209913,4.703134,79.883382,82.944606,85.860058,89.431487,94.169096
D->A,10.0,55.540947,2.568259,51.615327,53.878663,55.897821,57.334711,58.827949
D->W,10.0,84.796875,3.443847,78.437500,82.968750,85.468750,86.953125,89.531250


In [136]:
ccsa_from_feat_batch_size_4096_uneven_scores = get_score_combinations_new(
    method='ccsa',
    experiment_id='ccsa_batch_size',
    config_key='batch_size',
    config_value=4096,
)*100
ccsa_from_feat_batch_size_4096_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,72.218750,3.708099,65.781250,72.656250,73.593750,73.906250,75.156250
A->D,5.0,66.588921,5.836732,57.142857,65.306122,68.804665,69.387755,72.303207
W->A,5.0,49.917355,3.039507,46.882044,47.746056,49.286251,51.164538,54.507889
W->D,5.0,77.492711,5.188358,71.720117,74.344023,75.510204,82.215743,83.673469
D->A,5.0,48.384673,4.853837,42.261458,45.078888,48.271976,52.779865,53.531180
D->W,5.0,80.156250,5.312500,73.750000,75.625000,82.343750,82.500000,86.562500


## d-SNE

In [137]:
# dsne_scores = get_score_combinations(
#     suffix='dsne', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191104094606',
#     to_date='20191104124943'
# )*100
# dsne_scores.describe().T

dsne_scores = get_score_combinations_new(
    method='dsne',
    experiment_id='dsne_even',
)*100
dsne_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,82.250000,3.335855,76.875000,82.031250,82.343750,84.531250,85.468750
A->D,5.0,81.574344,1.561877,79.591837,80.466472,81.632653,83.090379,83.090379
W->A,5.0,61.307288,1.935638,58.865515,60.217881,61.645379,61.758077,64.049587
W->D,5.0,93.819242,1.463545,91.836735,93.586006,93.586006,94.169096,95.918367
D->A,5.0,61.134485,1.847341,58.677686,60.668670,60.781367,61.795642,63.749061
D->W,5.0,93.156250,1.261664,91.718750,91.875000,93.750000,94.062500,94.375000


In [138]:
# dsne_uneven_scores = get_score_combinations(
#     suffix='dsne', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191104152038',
#     to_date='20191104182550'
# )*100
# dsne_uneven_scores.describe().T

dsne_uneven_scores = get_score_combinations_new(
    method='dsne',
    experiment_id='dsne_uneven',
)*100
dsne_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.875000,3.706453,75.781250,76.875000,79.375000,83.437500,83.906250
A->D,5.0,80.408163,3.775482,74.927114,78.134111,81.924198,83.381924,83.673469
W->A,5.0,61.111946,0.951237,60.067618,60.368144,61.194591,61.457551,62.471826
W->D,5.0,88.279883,3.891883,83.381924,87.172012,87.755102,88.921283,94.169096
D->A,5.0,59.804658,0.668839,58.903080,59.541698,59.917355,59.917355,60.743802
D->W,5.0,90.281250,1.560154,87.812500,90.000000,90.781250,90.781250,92.031250


In [139]:
# dsne_large_uneven_scores = get_score_combinations(
#     suffix='dsne', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191105120356',
#     to_date='20191105154214'
# )*100
# dsne_large_uneven_scores.describe().T

dsne_large_uneven_scores = get_score_combinations_new(
    method='dsne',
    experiment_id='dsne_uneven_large',
)*100
dsne_large_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.062500,1.820832,77.031250,79.843750,80.781250,80.937500,81.718750
A->D,5.0,80.116618,2.511353,77.259475,78.425656,79.883382,81.341108,83.673469
W->A,5.0,58.219384,2.145921,55.409467,56.423742,59.429001,59.729527,60.105184
W->D,5.0,89.387755,3.725624,84.256560,88.046647,88.629738,92.419825,93.586006
D->A,5.0,58.422239,2.761963,55.785124,56.311044,57.325319,60.706236,61.983471
D->W,5.0,89.125000,2.752396,84.843750,88.593750,89.375000,90.625000,92.187500


In [140]:
# dsne_resnet_uneven_scores = get_score_combinations(
#     suffix='dsne', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191107085855',
#     to_date='20191107131237'
# )*100
# dsne_resnet_uneven_scores.describe().T


dsne_resnet_uneven_scores = get_score_combinations_new(
    method='dsne',
    experiment_id='dsne_uneven_resnet',
)*100
dsne_resnet_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,84.031250,1.820162,81.718750,83.281250,83.437500,85.468750,86.250000
A->D,5.0,85.072886,1.692469,83.673469,83.965015,84.256560,85.714286,87.755102
W->A,5.0,64.800902,2.708513,61.044328,64.425244,64.725770,65.138993,68.670173
W->D,5.0,91.020408,1.434213,89.795918,89.795918,90.962099,91.253644,93.294461
D->A,5.0,64.861007,1.711943,63.072878,63.486101,65.063862,65.289256,67.392938
D->W,5.0,89.406250,1.056278,88.281250,88.593750,89.375000,89.843750,90.937500


### d-SNE from features

In [141]:
dsne_from_feat_batch_size_16_uneven_scores = get_score_combinations_new(
    method='dsne',
    experiment_id='dsne_from_features',
)*100
dsne_from_feat_batch_size_16_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.531250,1.101535,76.093750,77.031250,77.500000,77.968750,79.062500
A->D,6.0,77.745384,2.512481,74.344023,76.457726,77.696793,78.717201,81.632653
W->A,5.0,58.534936,0.932130,57.513148,58.001503,58.489857,58.677686,59.992487
W->D,5.0,89.037901,3.725624,84.256560,86.588921,89.795918,90.670554,93.877551
D->A,5.0,58.309542,1.259265,56.649136,57.475582,58.489857,59.166041,59.767092
D->W,5.0,84.937500,1.172396,83.281250,84.375000,85.000000,85.781250,86.250000


## Dage

In [142]:
# dage_resnet_uneven_scores = get_score_combinations(
#     suffix='homebrew', 
#     domains=['A','W','D'], 
#     avg_types=['macro avg'],
#     from_date='20191113093959',
#     to_date='20191113143542'
# )*100
# dage_resnet_uneven_scores.describe().T

dage_resnet_uneven_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_uneven_resnet',
)*100
dage_resnet_uneven_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,86.531250,2.469047,83.281250,84.531250,87.656250,88.437500,88.750000
A->D,5.0,87.930029,1.951394,86.005831,86.588921,87.172012,89.212828,90.670554
W->A,5.0,63.080391,3.665057,57.400451,61.758077,63.936890,65.927874,66.378663
W->D,5.0,89.212828,3.022805,85.714286,87.755102,88.046647,91.253644,93.294461
D->A,5.0,62.193839,3.735912,55.634861,62.659654,63.861758,64.312547,64.500376
D->W,5.0,87.156250,1.989965,84.062500,86.250000,88.281250,88.437500,88.750000


### DAGE on logits
Varying batch_size

In [143]:
dage_logits_bs_16_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_logits_batch_size',
    config_key='batch_size',
    config_value=16,
)*100
dage_logits_bs_16_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,4.0,78.320312,1.136623,76.718750,78.007812,78.593750,78.906250,79.375000
A->D,5.0,78.075802,3.101909,74.927114,75.510204,77.259475,81.341108,81.341108
W->A,5.0,57.610819,1.481967,56.123216,56.160781,58.001503,58.151766,59.616829
W->D,5.0,84.548105,4.493013,80.466472,80.758017,82.798834,88.921283,89.795918
D->A,5.0,58.557476,2.234672,55.860255,56.461307,59.504132,60.405710,60.555973
D->W,5.0,82.625000,2.473986,78.437500,82.343750,83.750000,84.218750,84.375000


In [144]:
dage_logits_bs_64_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_logits_batch_size',
    config_key='batch_size',
    config_value=64,
)*100
dage_logits_bs_64_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,78.968750,1.039973,77.656250,78.593750,78.593750,79.687500,80.312500
A->D,5.0,77.842566,3.474162,73.760933,76.676385,76.676385,79.008746,83.090379
W->A,5.0,57.971450,1.724632,56.423742,56.536439,57.325319,59.316304,60.255447
W->D,5.0,80.816327,4.231927,76.384840,79.300292,79.591837,81.049563,87.755102
D->A,5.0,56.799399,1.581781,54.094666,56.912096,57.287754,57.513148,58.189331
D->W,5.0,79.093750,2.362942,77.656250,77.812500,78.281250,78.437500,83.281250


In [145]:
dage_logits_bs_256_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_logits_batch_size',
    config_key='batch_size',
    config_value=256,
)*100
dage_logits_bs_256_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.437500,1.044657,75.468750,75.468750,76.250000,77.187500,77.812500
A->D,5.0,77.900875,3.730184,74.635569,74.927114,76.967930,79.300292,83.673469
W->A,5.0,56.829452,2.908180,51.652893,57.813674,57.926371,58.189331,58.564989
W->D,5.0,77.609329,2.326889,74.344023,76.384840,78.425656,78.425656,80.466472
D->A,5.0,56.070624,0.468142,55.296769,55.972953,56.311044,56.311044,56.461307
D->W,5.0,75.781250,1.589609,73.593750,75.312500,75.312500,77.343750,77.343750


In [146]:
dage_logits_bs_1024_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_logits_batch_size',
    config_key='batch_size',
    config_value=1024,
)*100
dage_logits_bs_1024_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,74.531250,2.679129,69.843750,75.000000,75.312500,76.250000,76.250000
A->D,5.0,76.501458,4.042923,72.011662,72.886297,76.676385,79.883382,81.049563
W->A,5.0,56.446281,2.204344,53.005259,55.822690,56.912096,57.738542,58.752817
W->D,5.0,75.510204,4.930477,68.221574,73.469388,76.384840,78.425656,81.049563
D->A,5.0,55.567243,3.025430,50.826446,54.845980,56.085650,57.287754,58.790383
D->W,5.0,75.281250,0.920417,74.062500,74.531250,75.781250,75.937500,76.093750


In [147]:
dage_logits_bs_full_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_logits_batch_size',
    config_key='batch_size',
    config_value=4096,
)*100
dage_logits_bs_full_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,74.625000,2.332264,71.562500,73.437500,74.375000,76.250000,77.500000
A->D,5.0,73.586006,3.952561,67.055394,74.344023,74.344023,74.344023,77.842566
W->A,5.0,55.469572,1.895600,53.305785,54.432757,55.071375,56.273479,58.264463
W->D,5.0,75.335277,3.250435,72.886297,73.177843,73.469388,76.676385,80.466472
D->A,5.0,55.026296,2.934627,50.638618,53.643877,56.010518,56.761833,58.076634
D->W,5.0,74.062500,2.105030,70.937500,74.062500,74.218750,74.218750,76.875000


### DAGE full on embeds
Varying batch_size

In [148]:
dage_embeds_bs_8_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=8,
)*100
dage_embeds_bs_8_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.781250,0.941398,78.750000,79.218750,79.843750,79.843750,81.250000
A->D,5.0,78.775510,2.381052,76.384840,77.259475,77.842566,80.174927,82.215743
W->A,5.0,58.362134,0.628371,57.663411,58.001503,58.114200,58.865515,59.166041
W->D,5.0,82.448980,6.582081,74.635569,77.259475,82.798834,87.172012,90.379009
D->A,5.0,58.730278,1.526066,57.738542,57.888805,57.963937,58.677686,61.382419
D->W,5.0,85.125000,0.972021,84.218750,84.375000,84.687500,86.093750,86.250000


In [149]:
dage_embeds_bs_16_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=16,
)*100
dage_embeds_bs_16_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.812500,1.067772,78.906250,78.906250,79.375000,80.625000,81.250000
A->D,5.0,80.233236,3.764209,75.510204,79.008746,79.300292,81.632653,85.714286
W->A,5.0,58.685199,1.004629,57.212622,58.414726,58.564989,59.466566,59.767092
W->D,5.0,85.714286,1.303830,83.673469,85.422741,86.005831,86.297376,87.172012
D->A,5.0,57.896319,2.357591,55.747558,55.860255,57.062359,60.105184,60.706236
D->W,5.0,86.281250,1.617772,83.593750,86.093750,86.718750,87.343750,87.656250


In [150]:
dage_embeds_bs_32_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=32,
)*100
dage_embeds_bs_32_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.531250,2.227316,74.218750,76.406250,78.593750,78.593750,79.843750
A->D,5.0,79.650146,3.209646,76.676385,77.842566,78.425656,80.466472,84.839650
W->A,5.0,57.084899,1.765710,54.883546,55.447032,58.226897,58.414726,58.452292
W->D,5.0,81.574344,4.181413,75.218659,80.466472,82.507289,83.090379,86.588921
D->A,5.0,57.888805,1.322026,56.160781,57.400451,57.475582,58.940646,59.466566
D->W,5.0,82.531250,2.996173,80.156250,80.781250,81.562500,82.500000,87.656250


In [152]:
dage_embeds_bs_64_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=64,
)*100
dage_embeds_bs_64_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.750000,1.407985,74.375000,76.875000,77.187500,77.187500,78.125000
A->D,5.0,78.367347,2.799441,75.510204,76.676385,77.842566,79.008746,82.798834
W->A,5.0,57.137491,1.641111,54.958678,55.935387,57.851240,57.963937,58.978212
W->D,5.0,79.650146,2.776576,75.801749,78.717201,79.591837,80.758017,83.381924
D->A,5.0,57.768595,0.993328,56.498873,57.212622,57.700977,58.377160,59.053343
D->W,5.0,79.250000,3.082524,73.906250,79.687500,80.156250,80.781250,81.718750


In [153]:
dage_embeds_bs_128_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=128,
)*100
dage_embeds_bs_128_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.625000,0.845773,75.625000,75.937500,76.718750,77.187500,77.656250
A->D,5.0,78.717201,3.498542,74.927114,75.801749,78.425656,81.341108,83.090379
W->A,5.0,56.746807,1.442642,55.409467,55.747558,56.273479,57.325319,58.978212
W->D,5.0,78.717201,4.714566,72.011662,75.801749,80.174927,82.798834,82.798834
D->A,5.0,56.574005,2.197194,54.019534,54.658152,57.212622,57.663411,59.316304
D->W,5.0,76.468750,1.850755,74.531250,75.312500,75.937500,77.343750,79.218750


In [154]:
dage_embeds_bs_full_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_embeds_batch_size',
    config_key='batch_size',
    config_value=4096,
)*100
dage_embeds_bs_full_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,73.500000,1.146070,71.718750,73.281250,73.750000,73.906250,74.843750
A->D,5.0,74.577259,1.419319,72.886297,73.760933,74.635569,74.927114,76.676385
W->A,5.0,56.491360,1.725041,53.831705,55.747558,57.212622,57.550714,58.114200
W->D,5.0,74.985423,2.354126,71.428571,73.760933,76.093294,76.676385,76.967930
D->A,5.0,56.453794,1.270977,54.320060,56.461307,56.649136,57.362885,57.475582
D->W,5.0,75.281250,2.180233,72.343750,74.531250,74.687500,77.031250,77.812500


### DAGE full across on embeds
Varying loss alpha

In [155]:
dage_embeds_full_across_025_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_full_across_alpha',
    config_key='loss_alpha',
    config_value=0.25,
)*100
dage_embeds_full_across_025_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.687500,0.940101,75.781250,75.937500,76.406250,77.343750,77.968750
A->D,5.0,78.192420,3.255660,76.093294,76.676385,76.967930,77.259475,83.965015
W->A,5.0,57.137491,0.977070,55.972953,56.536439,57.250188,57.362885,58.564989
W->D,5.0,78.425656,3.564729,73.760933,76.093294,79.008746,80.466472,82.798834
D->A,5.0,56.904583,1.584321,54.695718,56.123216,57.062359,57.813674,58.827949
D->W,5.0,79.437500,3.069030,77.187500,77.343750,77.343750,81.406250,83.906250


In [156]:
dage_embeds_full_across_05_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_full_across_alpha',
    config_key='loss_alpha',
    config_value=0.5,
)*100
dage_embeds_full_across_05_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.781250,1.227334,76.093750,76.875000,78.437500,78.593750,78.906250
A->D,5.0,80.349854,2.731827,76.093294,80.174927,80.758017,81.049563,83.673469
W->A,5.0,57.851240,1.307536,55.597295,58.151766,58.226897,58.264463,59.015778
W->D,5.0,84.373178,4.575495,79.591837,81.341108,82.798834,87.463557,90.670554
D->A,5.0,57.385424,1.587036,55.597295,56.461307,56.761833,58.865515,59.241172
D->W,5.0,82.937500,2.082878,79.843750,82.031250,83.750000,83.750000,85.312500


In [157]:
dage_embeds_full_across_075_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_full_across_alpha',
    config_key='loss_alpha',
    config_value=0.75,
)*100
dage_embeds_full_across_075_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.187500,1.658754,77.968750,79.062500,80.468750,81.562500,81.875000
A->D,5.0,80.349854,2.382836,77.842566,79.008746,79.300292,81.924198,83.673469
W->A,5.0,59.338843,1.619820,57.212622,58.339594,59.353869,60.631104,61.157025
W->D,5.0,85.655977,3.608571,80.466472,84.548105,85.131195,88.629738,89.504373
D->A,5.0,60.090158,1.318177,58.564989,59.541698,59.579264,60.781367,61.983471
D->W,5.0,85.906250,3.789505,80.625000,85.000000,85.000000,88.281250,90.625000


### DAGE pair across on embeds
Varying loss alpha

In [158]:
dage_embeds_pair_across_025_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_pair_across_alpha',
    config_key='loss_alpha',
    config_value=0.25,
)*100
dage_embeds_pair_across_025_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.312500,0.732877,79.218750,80.000000,80.468750,80.781250,81.093750
A->D,5.0,80.816327,3.189723,76.093294,80.174927,80.466472,82.798834,84.548105
W->A,5.0,59.691961,3.706751,53.305785,59.691961,61.419985,61.607814,62.434260
W->D,5.0,90.204082,1.121597,88.629738,89.795918,90.087464,90.962099,91.545190
D->A,5.0,58.715252,1.547280,57.062359,57.700977,58.827949,58.865515,61.119459
D->W,5.0,88.968750,2.639192,84.843750,88.125000,89.531250,90.937500,91.406250


In [159]:
dage_embeds_pair_across_05_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_pair_across_alpha',
    config_key='loss_alpha',
    config_value=0.5,
)*100
dage_embeds_pair_across_05_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.531250,1.863899,78.906250,78.906250,80.468750,80.937500,83.437500
A->D,5.0,79.650146,2.363135,78.134111,78.134111,78.425656,79.883382,83.673469
W->A,5.0,60.368144,1.944730,57.475582,59.879790,60.180316,61.908340,62.396694
W->D,5.0,90.379009,4.737049,82.215743,90.962099,92.128280,92.128280,94.460641
D->A,5.0,60.743802,2.896340,56.686702,59.879790,60.443276,62.208866,64.500376
D->W,5.0,89.437500,2.311233,86.406250,88.125000,89.218750,91.718750,91.718750


In [160]:
dage_embeds_pair_across_075_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_pair_across_alpha',
    config_key='loss_alpha',
    config_value=0.75,
)*100
dage_embeds_pair_across_075_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.062500,1.182762,78.593750,79.687500,80.000000,80.156250,81.875000
A->D,5.0,79.125364,3.490029,74.344023,76.967930,79.883382,81.341108,83.090379
W->A,5.0,61.953418,0.422844,61.307288,61.758077,62.171300,62.171300,62.359128
W->D,5.0,91.603499,2.485840,89.795918,90.087464,90.962099,91.253644,95.918367
D->A,5.0,60.300526,1.880990,58.602554,58.640120,59.917355,61.344853,62.997746
D->W,5.0,91.187500,0.991911,90.312500,90.468750,90.625000,92.187500,92.343750


### DAGE MFA-like
Varying k

In [161]:
dage_embeds_across_knn_knn_1_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_mfa',
    config_key='connection_filter_param',
    config_value=1,
)*100
dage_embeds_across_knn_knn_1_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,80.531250,0.785924,79.375000,80.468750,80.468750,80.781250,81.562500
A->D,5.0,81.865889,1.717397,79.300292,81.341108,82.215743,82.507289,83.965015
W->A,5.0,61.810669,0.630053,61.081893,61.269722,62.021037,62.058603,62.622089
W->D,5.0,91.428571,1.407291,89.504373,90.379009,92.128280,92.419825,92.711370
D->A,5.0,61.555222,2.007501,59.579264,59.654395,61.682945,62.546957,64.312547
D->W,5.0,91.218750,0.984499,89.843750,90.781250,91.406250,91.562500,92.500000


In [162]:
dage_embeds_across_knn_knn_2_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_mfa',
    config_key='connection_filter_param',
    config_value=2,
)*100
dage_embeds_across_knn_knn_2_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.000000,2.249674,75.312500,79.062500,79.375000,79.843750,81.406250
A->D,5.0,80.583090,1.495147,79.008746,79.883382,79.883382,81.341108,82.798834
W->A,5.0,61.074380,1.224609,59.579264,60.217881,61.081893,61.870774,62.622089
W->D,5.0,91.078717,3.130549,86.588921,89.795918,90.962099,93.877551,94.169096
D->A,5.0,62.268971,1.532756,60.180316,61.682945,61.983471,63.410969,64.087153
D->W,5.0,90.343750,1.980742,87.187500,89.687500,91.093750,91.718750,92.031250


### DAGE d-SNE-like
Varying k

In [163]:
dage_embeds_across_kfn_knn_1_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_dsne',
    config_key='connection_filter_param',
    config_value=1,
)*100
dage_embeds_across_kfn_knn_1_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.375000,1.502764,78.125000,78.281250,79.218750,79.375000,81.875000
A->D,5.0,81.574344,2.698959,77.551020,80.174927,82.507289,83.673469,83.965015
W->A,5.0,61.547708,1.028643,60.293013,60.743802,61.607814,62.471826,62.622089
W->D,5.0,91.661808,2.067711,89.504373,89.795918,91.545190,93.294461,94.169096
D->A,5.0,61.855748,1.271310,60.368144,61.232156,61.307288,63.072878,63.298272
D->W,5.0,89.656250,2.539723,87.187500,87.500000,88.906250,92.187500,92.500000


In [164]:
dage_embeds_across_kfn_knn_2_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_dsne',
    config_key='connection_filter_param',
    config_value=2,
)*100
dage_embeds_across_kfn_knn_2_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,79.343750,0.744446,78.125000,79.218750,79.531250,79.843750,80.000000
A->D,5.0,80.408163,0.696055,79.591837,79.883382,80.466472,80.758017,81.341108
W->A,4.0,62.105560,0.638526,61.607814,61.664162,61.908340,62.349737,62.997746
W->D,5.0,91.895044,2.086128,89.212828,91.545190,91.545190,92.128280,95.043732
D->A,5.0,61.006762,1.051841,59.278738,60.818933,61.344853,61.645379,61.945905
D->W,5.0,90.750000,1.454888,89.218750,89.687500,90.468750,91.562500,92.812500


### DAGE on aux dense layer
Varying embedding size

In [165]:
dage_aux_dense_16_bs_full_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_vary_emb_size',
    config_key='aux_dense_size',
    config_value=16,
)*100
dage_aux_dense_16_bs_full_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,3.0,74.531250,0.869963,73.593750,74.140625,74.687500,75.000000,75.312500
A->D,2.0,74.344023,1.649229,73.177843,73.760933,74.344023,74.927114,75.510204
W->A,3.0,57.450538,1.094791,56.423742,56.874530,57.325319,57.963937,58.602554
W->D,3.0,75.218659,2.314068,73.469388,73.906706,74.344023,76.093294,77.842566
D->A,3.0,57.763586,1.796229,55.860255,56.930879,58.001503,58.715252,59.429001
D->W,3.0,74.635417,1.301041,73.593750,73.906250,74.218750,75.156250,76.093750


In [166]:
dage_aux_dense_31_bs_full_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_vary_emb_size',
    config_key='aux_dense_size',
    config_value=31,
)*100
dage_aux_dense_31_bs_full_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,74.593750,1.430349,73.125000,73.437500,74.375000,75.468750,76.562500
A->D,5.0,75.568513,3.836894,71.428571,73.760933,73.760933,77.842566,81.049563
W->A,5.0,54.740796,5.062978,45.830203,55.634861,56.724267,57.438017,58.076634
W->D,5.0,75.160350,5.605710,67.055394,72.594752,76.967930,77.259475,81.924198
D->A,5.0,56.671675,1.745615,54.320060,56.311044,56.536439,56.987228,59.203606
D->W,5.0,73.562500,3.268950,67.968750,73.750000,74.687500,75.000000,76.406250


In [167]:
dage_aux_dense_64_bs_full_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_vary_emb_size',
    config_key='aux_dense_size',
    config_value=64,
)*100
dage_aux_dense_64_bs_full_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,73.187500,1.710206,71.562500,71.718750,72.656250,74.843750,75.156250
A->D,5.0,75.276968,3.752901,69.970845,73.177843,76.384840,77.259475,79.591837
W->A,5.0,56.829452,1.623692,54.733283,55.897821,56.836965,57.738542,58.940646
W->D,5.0,76.967930,3.733600,73.760933,74.052478,75.510204,79.008746,82.507289
D->A,5.0,56.468820,1.337520,54.583020,55.597295,56.949662,57.513148,57.700977
D->W,5.0,74.812500,1.067772,73.906250,74.062500,74.531250,75.000000,76.562500


In [60]:
# The above experiments used all data every batch. We saw this to be suboptimal in the logits experiments, so here is a sredo with bs=16

In [168]:
dage_aux_dense_16_bs_16_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_aux_dense_low_bs',
    config_key='aux_dense_size',
    config_value=16,
)*100
dage_aux_dense_16_bs_16_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.468750,1.267456,75.000000,75.312500,76.718750,77.500000,77.812500
A->D,5.0,77.784257,2.889094,74.344023,76.384840,76.676385,80.174927,81.341108
W->A,5.0,57.145004,1.789289,54.244929,56.987228,57.287754,58.377160,58.827949
W->D,5.0,78.134111,3.220222,74.052478,76.967930,77.551020,79.300292,82.798834
D->A,5.0,57.129977,1.601376,54.770849,56.686702,56.987228,58.339594,58.865515
D->W,5.0,76.843750,1.186883,75.781250,75.937500,76.406250,77.500000,78.593750


In [169]:
dage_aux_dense_31_bs_16_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_aux_dense_low_bs',
    config_key='aux_dense_size',
    config_value=31,
)*100
dage_aux_dense_31_bs_16_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.843750,0.719429,75.937500,76.562500,76.562500,77.500000,77.656250
A->D,5.0,78.600583,2.660899,75.801749,76.967930,78.425656,79.008746,82.798834
W->A,5.0,57.520661,1.154620,56.123216,56.423742,58.114200,58.377160,58.564989
W->D,5.0,79.183673,5.286543,72.594752,76.093294,78.134111,84.256560,84.839650
D->A,5.0,56.784373,1.150088,54.845980,56.686702,57.175056,57.513148,57.700977
D->W,5.0,77.906250,1.773283,75.625000,77.187500,77.812500,78.437500,80.468750


In [170]:
dage_aux_dense_64_bs_16_scores = get_score_combinations_new(
    method='dage',
    experiment_id='dage_aux_dense_low_bs',
    config_key='aux_dense_size',
    config_value=64,
)*100
dage_aux_dense_64_bs_16_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,76.437500,2.527679,72.812500,74.843750,77.500000,78.281250,78.750000
A->D,5.0,78.425656,3.468040,75.218659,76.384840,76.676385,80.174927,83.673469
W->A,5.0,56.491360,1.429326,54.507889,55.484598,57.099925,57.588279,57.776108
W->D,5.0,79.766764,2.800959,76.967930,77.551020,79.591837,80.758017,83.965015
D->A,5.0,57.445530,0.446385,56.912096,57.137491,57.438017,57.700977,58.039068
D->W,5.0,77.625000,1.695150,75.312500,76.875000,77.656250,78.437500,79.843750


### DAGE with class-wise attention
Even loss weights (source and target entropy both weighted)
Varying loss alpha

In [171]:
dage_cw_attention_025_scores = get_score_combinations_new(
    method='dage_a',
    experiment_id='dage_attention_even',
    config_key='loss_alpha',
    config_value=0.25,
)*100
dage_cw_attention_025_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.125000,0.407450,76.718750,76.718750,77.187500,77.343750,77.656250
A->D,5.0,77.434402,1.280809,76.093294,76.676385,76.967930,78.134111,79.300292
W->A,5.0,60.646131,1.845621,58.414726,59.203606,60.894065,61.795642,62.922615
W->D,5.0,92.769679,2.425263,88.921283,92.128280,93.294461,94.460641,95.043732
D->A,5.0,61.029301,1.486721,59.579264,59.992487,60.330579,62.396694,62.847483
D->W,5.0,91.718750,2.058116,88.750000,91.406250,91.875000,92.031250,94.531250


In [172]:
dage_cw_attention_050_scores = get_score_combinations_new(
    method='dage_a',
    experiment_id='dage_attention_even',
    config_key='loss_alpha',
    config_value=0.5,
)*100
dage_cw_attention_050_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,77.031250,1.668293,75.000000,76.093750,76.562500,78.750000,78.750000
A->D,5.0,76.443149,3.255660,71.137026,75.801749,77.259475,79.008746,79.008746
W->A,5.0,61.592787,1.211633,60.180316,60.555973,61.682945,62.697220,62.847483
W->D,5.0,92.069971,1.492301,89.504373,92.128280,92.711370,92.711370,93.294461
D->A,5.0,60.443276,1.843977,58.151766,58.865515,61.044328,61.795642,62.359128
D->W,5.0,91.125000,1.709492,89.218750,90.156250,90.937500,91.562500,93.750000


In [173]:
dage_cw_attention_075_scores = get_score_combinations_new(
    method='dage_a',
    experiment_id='dage_attention_even',
    config_key='loss_alpha',
    config_value=0.75,
)*100
dage_cw_attention_075_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
A->W,5.0,78.531250,1.028168,77.656250,77.812500,78.125000,78.906250,80.156250
A->D,5.0,76.967930,2.030377,73.760933,76.384840,77.551020,78.134111,79.008746
W->A,5.0,61.284748,1.651312,58.903080,60.518407,61.607814,62.171300,63.223140
W->D,5.0,91.195335,0.859935,90.379009,90.379009,91.253644,91.545190,92.419825
D->A,5.0,61.141998,1.892993,58.039068,61.006762,61.419985,62.283997,62.960180
D->W,5.0,90.718750,1.213329,89.687500,89.843750,90.312500,91.093750,92.656250


## Summary

In [174]:
def present(df, name):
    df = df.describe().T[['mean','std']]
    tot_mean = df.mean()[['mean']]
    
    toStr = lambda s: f'{s:.2f}'
    df = df['mean'].map(toStr) + ' ±' + df['std'].map(toStr)
    df = pd.concat([df,tot_mean[['mean']]])
    return pd.DataFrame(df, columns=[name]).T


def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

df_all_from_img_vgg16 = pd.concat([
    present(tune_source_no_aug_scores, 'FT (source only, w/o aug, VGG16)'),
    present(tune_source_scores, 'FT (source only, w. aug, VGG16)'),
    present(tune_both_scores, 'FT (source and target, w. aug, VGG16)'),
    present(multitask_scores, 'Multitask, w. aug, VGG16'),
    present(ccsa_scores, 'CCSA (even, w. aug, VGG16)'),
    present(ccsa_uneven_scores, 'CCSA (uneven, w. aug, VGG16)'),
    present(dsne_scores, 'd-SNE (even, w. aug, VGG16)'),
    present(dsne_uneven_scores, 'd-SNE (uneven, w. aug, VGG16)'),
#     present(dsne_large_uneven_scores, 'd-SNE (large, w. aug, uneven, VGG16)'),
#     present(dage_logits_bs_16_scores, 'DAGE (uneven, w/o. aug, VGG16, bs 16)'),
#     present(dage_logits_bs_full_scores, 'DAGE (uneven, w/o. aug, VGG16, bs full)'),
#     present(ccsa_resnet_uneven_scores, 'CCSA (uneven, ResNet101_v2)'),
#     present(dsne_resnet_uneven_scores, 'd-SNE (uneven, ResNet101_v2)'),
#     present(dage_resnet_uneven_scores, 'DAGE (uneven, ResNet101_v2)'),
])

df_all_from_feat_vgg16 = pd.concat([
    present(multitask_from_feat_scores, 'Multitask (w/o. aug, VGG16, bs 16)'),
    present(ccsa_from_feat_batch_size_16_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs 16)'),
    present(ccsa_from_feat_batch_size_32_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs 32)'),
    present(ccsa_from_feat_batch_size_64_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs 64)'),
    present(ccsa_from_feat_batch_size_128_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs 128)'),
    present(ccsa_from_feat_batch_size_256_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs 256)'),
    present(ccsa_from_feat_batch_size_4096_uneven_scores, 'CCSA (uneven, w/o. aug, VGG16, bs full)'),
    present(dsne_from_feat_batch_size_16_uneven_scores, 'd-SNE (uneven, w/o. aug, VGG16, bs 16)'),
    present(dage_aux_dense_16_bs_full_scores, 'DAGE (uneven, aux dense 16, w/o. aug, VGG16, bs full)'),
    present(dage_aux_dense_31_bs_full_scores, 'DAGE (uneven, aux dense 31, w/o. aug, VGG16, bs full)'),
    present(dage_aux_dense_64_bs_full_scores, 'DAGE (uneven, aux dense 64, w/o. aug, VGG16, bs full)'),
    present(dage_aux_dense_16_bs_16_scores, 'DAGE (uneven, aux dense 16, w/o. aug, VGG16, bs 16)'),
    present(dage_aux_dense_31_bs_16_scores, 'DAGE (uneven, aux dense 31, w/o. aug, VGG16, bs 16)'),
    present(dage_aux_dense_64_bs_16_scores, 'DAGE (uneven, aux dense 64, w/o. aug, VGG16, bs 16)'),
    present(dage_logits_bs_16_scores, 'DAGE (uneven, logits, w/o. aug, VGG16, bs 16)'),
    present(dage_logits_bs_64_scores, 'DAGE (uneven, logits, w/o. aug, VGG16, bs 64)'),
    present(dage_logits_bs_256_scores, 'DAGE (uneven, logits, w/o. aug, VGG16, bs 256)'),
    present(dage_logits_bs_1024_scores, 'DAGE (uneven, logits, w/o. aug, VGG16, bs 1024)'),
    present(dage_logits_bs_full_scores, 'DAGE (uneven, logits, w/o. aug, VGG16, bs full)'),
    present(dage_embeds_bs_8_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs 8)'),
    present(dage_embeds_bs_16_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_bs_32_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs 32)'),
    present(dage_embeds_bs_64_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs 64)'),
    present(dage_embeds_bs_128_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs 128)'),
    present(dage_embeds_bs_full_scores, 'DAGE (uneven, embeds, full, w/o. aug, VGG16, bs full)'),
    present(dage_embeds_full_across_025_scores, 'DAGE (uneven, embeds, full across, alpha=0.25 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_full_across_05_scores,  'DAGE (uneven, embeds, full across, alpha=0.50 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_full_across_075_scores, 'DAGE (uneven, embeds, full across, alpha=0.75 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_pair_across_025_scores, 'DAGE (uneven, embeds, pair across, alpha=0.25 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_pair_across_05_scores,  'DAGE (uneven, embeds, pair across, alpha=0.50 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_pair_across_075_scores, 'DAGE (uneven, embeds, pair across, alpha=0.75 w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_across_knn_knn_1_scores, 'DAGE (uneven, embeds, full across, knn, knn, k=1, w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_across_knn_knn_2_scores, 'DAGE (uneven, embeds, full across, knn, knn, k=2, w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_across_kfn_knn_1_scores, 'DAGE (uneven, embeds, full across, kfn, knn, k=1, w/o. aug, VGG16, bs 16)'),
    present(dage_embeds_across_kfn_knn_2_scores, 'DAGE (uneven, embeds, full across, kfn, knn, k=2, w/o. aug, VGG16, bs 16)'),
    present(dage_cw_attention_025_scores, 'DAGE (even, embeds, full across, cw attention, alpha=0.25 w/o. aug, VGG16, bs 16)'),
    present(dage_cw_attention_050_scores, 'DAGE (even, embeds, full across, cw attention, alpha=0.5 w/o. aug, VGG16, bs 16)'),
    present(dage_cw_attention_075_scores, 'DAGE (even, embeds, full across, cw attention, alpha=0.75 w/o. aug, VGG16, bs 16)'),
])

df_all_from_img_renset101 = pd.concat([
    present(ccsa_resnet_uneven_scores, 'CCSA (uneven, ResNet101_v2)'),
    present(dsne_resnet_uneven_scores, 'd-SNE (uneven, ResNet101_v2)'),
    present(dage_resnet_uneven_scores, 'DAGE (uneven, ResNet101_v2)'),
])

In [175]:
df_all_from_img_vgg16.style.apply(highlight_max)

,A->W,A->D,W->A,W->D,D->A,D->W,mean
"FT (source only, w/o aug, VGG16)",52.66 ±1.25,56.96 ±3.21,42.34 ±1.47,98.57 ±0.57,36.84 ±2.34,91.03 ±2.01,63.0683
"FT (source only, w. aug, VGG16)",55.34 ±1.86,59.11 ±1.69,40.91 ±2.01,98.75 ±0.68,40.82 ±1.71,93.62 ±1.20,64.7596
"FT (source and target, w. aug, VGG16)",60.06 ±3.52,65.54 ±3.10,51.44 ±1.76,99.23 ±0.58,52.77 ±2.71,94.56 ±2.71,70.5994
"Multitask, w. aug, VGG16",81.69 ±1.94,80.23 ±3.69,62.91 ±1.67,96.33 ±1.91,62.15 ±1.91,93.38 ±1.06,79.4464
"CCSA (even, w. aug, VGG16)",82.50 ±2.72,82.27 ±3.61,59.94 ±2.55,93.24 ±2.52,60.11 ±1.92,92.44 ±2.33,78.4167
"CCSA (uneven, w. aug, VGG16)",83.47 ±1.46,82.92 ±2.25,60.92 ±3.09,93.94 ±1.65,60.83 ±2.82,92.53 ±2.62,79.1003
"d-SNE (even, w. aug, VGG16)",82.25 ±3.34,81.57 ±1.56,61.31 ±1.94,93.82 ±1.46,61.13 ±1.85,93.16 ±1.26,78.8736
"d-SNE (uneven, w. aug, VGG16)",79.88 ±3.71,80.41 ±3.78,61.11 ±0.95,88.28 ±3.89,59.80 ±0.67,90.28 ±1.56,76.6268


In [176]:
df_all_from_feat_vgg16.style.apply(highlight_max)

,A->W,A->D,W->A,W->D,D->A,D->W,mean
"Multitask (w/o. aug, VGG16, bs 16)",77.34 ±1.61,77.20 ±2.07,61.60 ±1.39,93.00 ±2.51,62.23 ±0.68,91.34 ±0.83,77.1205
"CCSA (uneven, w/o. aug, VGG16, bs 16)",79.25 ±1.81,78.48 ±1.64,59.56 ±0.79,90.03 ±3.43,58.77 ±2.07,90.53 ±0.92,76.1032
"CCSA (uneven, w/o. aug, VGG16, bs 32)",79.31 ±1.59,78.08 ±2.49,59.47 ±1.36,90.15 ±2.16,59.12 ±1.78,90.62 ±1.08,76.1257
"CCSA (uneven, w/o. aug, VGG16, bs 64)",79.71 ±2.31,79.14 ±3.45,61.36 ±1.11,88.77 ±2.19,62.50 ±1.69,90.69 ±2.12,77.0291
"CCSA (uneven, w/o. aug, VGG16, bs 128)",78.00 ±1.61,76.15 ±3.36,57.98 ±2.10,87.70 ±2.21,57.93 ±1.21,88.16 ±2.79,74.3183
"CCSA (uneven, w/o. aug, VGG16, bs 256)",77.00 ±2.09,76.38 ±1.68,53.62 ±3.49,86.21 ±4.70,55.54 ±2.57,84.80 ±3.44,72.259
"CCSA (uneven, w/o. aug, VGG16, bs full)",72.22 ±3.71,66.59 ±5.84,49.92 ±3.04,77.49 ±5.19,48.38 ±4.85,80.16 ±5.31,65.7931
"d-SNE (uneven, w/o. aug, VGG16, bs 16)",77.53 ±1.10,77.75 ±2.51,58.53 ±0.93,89.04 ±3.73,58.31 ±1.26,84.94 ±1.17,74.3494
"DAGE (uneven, aux dense 16, w/o. aug, VGG16, bs full)",74.53 ±0.87,74.34 ±1.65,57.45 ±1.09,75.22 ±2.31,57.76 ±1.80,74.64 ±1.30,68.9906
"DAGE (uneven, aux dense 31, w/o. aug, VGG16, bs full)",74.59 ±1.43,75.57 ±3.84,54.74 ±5.06,75.16 ±5.61,56.67 ±1.75,73.56 ±3.27,68.3829


In [177]:
df_all_from_img_renset101.style.apply(highlight_max)

,A->W,A->D,W->A,W->D,D->A,D->W,mean
"CCSA (uneven, ResNet101_v2)",86.69 ±1.84,84.99 ±1.70,66.47 ±2.01,93.35 ±2.06,63.48 ±4.04,92.19 ±1.18,81.1936
"d-SNE (uneven, ResNet101_v2)",84.03 ±1.82,85.07 ±1.69,64.80 ±2.71,91.02 ±1.43,64.86 ±1.71,89.41 ±1.06,79.8655
"DAGE (uneven, ResNet101_v2)",86.53 ±2.47,87.93 ±1.95,63.08 ±3.67,89.21 ±3.02,62.19 ±3.74,87.16 ±1.99,79.3508


# Utils

In [ ]:
# restructure former runs
def reorganize(from_date, to_date, suffix, experiment_id='your_id', description='your_description_here'):

    RUNS_DIR_OLD = RUNS_DIR / '..' / 'runs_old'

    for item in RUNS_DIR_OLD.glob('*'):
        parts = item.name.split('_')
        if len(parts)<3:
            continue
        timestamp = parts[0]
        src = parts[1]
        tgt = parts[2]
        method = '_'.join(parts[3:])
        
        if (item.is_dir() 
            and item.name.endswith(suffix)
            and int(item.name.split('_')[0]) >= int(from_date)
            and int(item.name.split('_')[0]) <= int(to_date)):

            seed = load_json(item / 'config.json')['seed']

            new_method_dir = RUNS_DIR / method / experiment_id 
            new_run_dir = new_method_dir / '{}{}_{}_{}'.format(src,tgt,seed,timestamp)
            
            try:
                shutil.copytree(item, new_run_dir)
            except:
                pass
            
            new_run_dir.mkdir(parents=True, exist_ok=True)

            with open(new_method_dir / 'description.txt', 'a') as f:
                f.write(description)


In [ ]:
reorganize(
    from_date='20191014123846',
    to_date='20191014162536',
    suffix='tune_source',
    experiment_id='tune_source_no_aug',
    description='In this experiment, we tune a VGG16-network pretrained on ImageNet with all available source data. The target data is used for validation (during training) and test. No augmentation was used'
)

reorganize(
    from_date='20191022103424',
    to_date='20191022142437',
    suffix='tune_source',
    experiment_id='tune_source_with_aug',
    description='In this experiment, we tune a VGG16-network pretrained on ImageNet with all available source data. The target data is used for validation (during training) and test. Augmentation was used'
)

reorganize(
    from_date='20191023072318',
    to_date='20191023123426',
    suffix='tune_target',
    experiment_id='tune_target_with_aug',
    description='Fine tune target from ImageNet model that was already fine-tuned on source. Augmentation was used. No batchnorm was used.'
)

reorganize(
    from_date='20191025141713',
    to_date='20191025170306',
    suffix='ccsa',
    experiment_id='ccsa_without_batchnorn',
    description='CCSA method without batchnorm. In this experiment, we saw stability issues for loss_alpha over 0.01.'
)

reorganize(
    from_date='20191101173414',
    to_date='20191101191220',
    suffix='ccsa',
    experiment_id='ccsa_with_batchnorn_alpha_0',
    description='CCSA method with batchnorm and loss_alpha=0. This amounts to a multi-task learning setup.'
)

reorganize(
    from_date='20191101150707',
    to_date='20191101171905',
    suffix='ccsa',
    experiment_id='ccsa_with_batchnorn_alpha_0.25',
    description='CCSA method with batchnorm and loss_alpha=0.25.'
)

reorganize(
    from_date='20191104152050',
    to_date='20191104185818',
    suffix='ccsa',
    experiment_id='ccsa_uneven',
    description='CCSA method with batchnorm and loss_alpha=0.25. Here, we only weight the target softmax (source softmax not used).'
)

reorganize(
    from_date='20191106083058',
    to_date='20191106144631',
    suffix='ccsa',
    experiment_id='ccsa_resnet_uneven',
    description='CCSA method with batchnorm and loss_alpha=0.25. Here, we only weight the target softmax (source softmax not used). ResNet features where used.'
)

reorganize(
    from_date='20191104094606',
    to_date='20191104124943',
    suffix='dsne',
    experiment_id='dsne_even',
    description='DSNE method using both source and target softmax loss'
)

reorganize(
    from_date='20191104152038',
    to_date='20191104182550',
    suffix='dsne',
    experiment_id='dsne_uneven',
    description='DSNE method using only target softmax loss'
)

reorganize(
    from_date='20191105120356',
    to_date='20191105154214',
    suffix='dsne',
    experiment_id='dsne_uneven_large',
    description='DSNE method using only target softmax loss. Larger dimensionality was used for dense and embedding layers'
)

reorganize(
    from_date='20191107085855',
    to_date='20191107131237',
    suffix='dsne',
    experiment_id='dsne_uneven_resnet',
    description='DSNE method using only target softmax loss. ResNet features where used.'
)

reorganize(
    from_date='20191113093959',
    to_date='20191113143542',
    suffix='homebrew',
    experiment_id='dage_uneven_resnet',
    description='DAGE method using only target softmax loss. ResNet features where used.'
)